
Step 1. [Download history price and clean data](/notebooks/forex/hawkeye/1_download.ipynb)

Step 2. [Use monte carlo simulate each stock's movement for 10000 times](/notebooks/forex/hawkeye/2_process.ipynb)

**Step 3. Apply some heuristic rules to give stock recommendation**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('/opt/hawkeye')
from asx import *

this_week=20190607
last_week=20190531
path = f'/opt/hawkeye/data/{this_week}/result.csv'
last_path = f'/opt/hawkeye/data/{last_week}/result.csv'

def week_return_avg(df):
    return round(df['week_return'].dropna().sum()/len(df['week_return'].dropna()),4)

df = pd.read_csv(path,
                 usecols=[
                     'code', 'last_date', 'start price', 'sim_mean',
                     'sim_diff', 'VaR 99%', 'VaR 99% Percent', 'volume_mean',
                     'return_mean', 'return_sigma'
                 ],
                 index_col='code')
last_df = pd.read_csv(last_path,
                      usecols=[
                          'code', 'last_date', 'start price', 'sim_mean',
                          'sim_diff', 'VaR 99%', 'VaR 99% Percent',
                          'volume_mean', 'return_mean', 'return_sigma'
                      ],
                      index_col='code')

print(f'Total stock number = {len(df)}')

# remove bottom volume and bottom price
volume_threshold = df['volume_mean'].quantile(0.1)
df = df[(df['volume_mean'] > volume_threshold) & (df['start price'] > 0.05)]

df['return'] = round(df['sim_diff'] / df['start price'] * 100, 3)
df['return_rank'] = round(df['return'].rank(pct=True) * 100, 3)
df['risk_rank'] = round(df['VaR 99% Percent'].rank(pct=True) * 100, 3)
df['volume_rank'] = round(df['volume_mean'].rank(pct=True) * 100, 3)

df['last_return'] = round(last_df['sim_diff'] / last_df['start price'] * 100,3)
df['return_increase'] = df['return'] - df['last_return']
df['week_return'] = round(
    (df['start price'] - last_df['start price']) / df['start price'] * 100, 3)
df.drop(columns=['sim_diff', 'sim_mean', 'VaR 99%','VaR 99% Percent', 'volume_mean'], inplace=True)

print(f'Total stock number after bottom volume & price removed = {len(df)}')
print('Price 70%% pricetile = %s' % df['start price'].quantile(0.7))
print('Sim return 90%% percentile = %s' % df['return'].quantile(0.9))
df.sort_values(by='return', ascending=False).head(20)

Total stock number = 1495
Total stock number after bottom volume & price removed = 943
Price 70% pricetile = 2.02
Sim return 90% percentile = 1.0394000000000005


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
OCC,2019-06-07,0.405,0.033110,0.365770,7.060,100.000,100.000,89.714,NaN,NaN,NaN
FTC,2019-06-07,0.145,0.024464,0.190723,4.869,99.894,99.894,71.898,5.325,-0.456,-6.897
FEX,2019-06-07,0.115,0.021486,0.093235,4.226,99.788,96.607,98.727,3.991,0.235,8.696
8CO,2019-06-07,0.160,0.019369,0.075834,3.857,99.682,86.426,47.826,3.486,0.371,3.125
LTR,2019-06-07,0.105,0.018831,0.094523,3.855,99.576,97.031,99.364,3.442,0.413,0.000
ANO,2019-06-07,6.480,0.019508,0.068453,3.785,99.470,82.185,25.027,4.132,-0.347,-4.321
WZR,2019-06-07,0.135,0.016248,0.101867,3.264,99.364,98.621,96.182,3.375,-0.111,-11.111
ISX,2019-06-07,0.650,0.016403,0.056279,3.232,99.258,66.066,85.896,3.123,0.109,4.615
NET,2019-06-07,0.096,0.016081,0.086080,2.954,99.152,95.122,90.244,2.761,0.193,9.375


In [13]:
# high mean return 
high_return=df.sort_values(by='return', ascending=False).head(10)
print('High return last week performance avg. = %s%%'% week_return_avg(high_return))
high_return

High return last week performance avg. = 0.7536%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
OCC,2019-06-07,0.405,0.033110,0.365770,7.060,100.000,100.000,89.714,NaN,NaN,NaN
FTC,2019-06-07,0.145,0.024464,0.190723,4.869,99.894,99.894,71.898,5.325,-0.456,-6.897
FEX,2019-06-07,0.115,0.021486,0.093235,4.226,99.788,96.607,98.727,3.991,0.235,8.696
8CO,2019-06-07,0.160,0.019369,0.075834,3.857,99.682,86.426,47.826,3.486,0.371,3.125
LTR,2019-06-07,0.105,0.018831,0.094523,3.855,99.576,97.031,99.364,3.442,0.413,0.000
ANO,2019-06-07,6.480,0.019508,0.068453,3.785,99.470,82.185,25.027,4.132,-0.347,-4.321
WZR,2019-06-07,0.135,0.016248,0.101867,3.264,99.364,98.621,96.182,3.375,-0.111,-11.111
ISX,2019-06-07,0.650,0.016403,0.056279,3.232,99.258,66.066,85.896,3.123,0.109,4.615
NET,2019-06-07,0.096,0.016081,0.086080,2.954,99.152,95.122,90.244,2.761,0.193,9.375


In [14]:
# high price

high_price=df[df['start price']>1].sort_values(by='return', ascending=False).head(10)
print('High price last week performance avg. = %s%%'% week_return_avg(high_price))
high_price

High price last week performance avg. = -6.0008%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
ANO,2019-06-07,6.48,0.019508,0.068453,3.785,99.470,82.185,25.027,4.132,-0.347,-4.321
DUB,2019-06-07,1.12,0.012231,0.058187,2.318,98.409,75.610,67.762,2.720,-0.402,-10.714
Z1P,2019-06-07,3.03,0.011510,0.045251,2.310,98.303,56.628,88.547,2.473,-0.163,-5.611
BUB,2019-06-07,1.08,0.010815,0.056006,2.099,97.561,72.216,95.652,2.314,-0.215,-15.278
CIA,2019-06-07,2.93,0.010259,0.040097,1.960,96.925,52.492,52.598,2.150,-0.190,-8.532
CRD,2019-06-07,1.72,0.009677,0.045503,1.866,96.713,57.052,50.053,1.849,0.017,1.744
5GN,2019-06-07,1.17,0.009048,0.045935,1.720,96.182,63.309,58.218,2.119,-0.399,-14.103
JIN,2019-06-07,17.25,0.008538,0.028880,1.692,96.076,31.813,53.022,1.867,-0.175,-4.348
AD8,2019-06-07,7.76,0.008159,0.025422,1.603,95.652,21.951,40.403,1.531,0.072,0.773


In [16]:
# highest price

highest_price=df[df['return']>0.6].sort_values(by='start price', ascending=False).head(10)
print('Highest price last week performance avg. = %s%%'% week_return_avg(highest_price))
highest_price

Highest price last week performance avg. = -0.3658%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
XRO,2019-06-07,59.29,0.003621,0.021198,0.728,83.245,22.269,61.824,0.664,0.064,-0.877
MFG,2019-06-07,45.83,0.005556,0.016239,1.083,90.986,9.438,65.536,1.195,-0.112,5.237
CUV,2019-06-07,31.69,0.004383,0.035006,0.771,84.942,49.417,31.601,1.301,-0.530,-8.836
ALU,2019-06-07,31.32,0.003385,0.028847,0.664,80.859,44.008,65.005,0.737,-0.073,1.309
JBH,2019-06-07,27.72,0.003262,0.013540,0.655,80.488,7.953,70.201,0.646,0.009,-1.443
WTC,2019-06-07,26.92,0.003885,0.028106,0.743,83.828,39.449,71.050,0.685,0.058,9.510
APX,2019-06-07,26.71,0.006893,0.032860,1.245,93.160,44.857,72.216,1.524,-0.279,-4.380
APT,2019-06-07,24.17,0.006590,0.039193,1.286,93.955,52.704,85.472,1.276,0.010,0.538
MND,2019-06-07,19.00,0.003074,0.015133,0.619,78.791,11.135,55.037,0.647,-0.028,-0.368


In [40]:
# low risk
low_risk = df.sort_values(by='risk_rank', ascending=True).head(20).sort_values(by='return', ascending=False)
print('Low risk last week performance avg. = %s%%'% week_return_avg(low_risk))
low_risk

Low risk last week performance avg. = 1.2228%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
ASX,2019-06-07,79.290,0.002558,0.009035,0.495,73.277,1.485,54.719,0.408,0.087,5.057
SHL,2019-06-07,26.480,0.002171,0.009002,0.411,68.346,1.803,76.352,0.380,0.031,1.246
AMC,2019-06-06,15.770,0.001996,0.008873,0.402,67.603,1.697,93.213,0.438,-0.036,-1.332
TCL,2019-06-07,14.260,0.001962,0.007889,0.380,66.384,0.954,95.228,0.369,0.011,2.314
CMW,2019-06-07,1.190,0.001858,0.008106,0.356,64.210,1.273,93.531,0.356,0.000,2.941
AST,2019-06-07,1.855,0.001793,0.007959,0.342,62.831,1.166,94.380,0.316,0.026,4.043
CMA,2019-06-07,2.740,0.001754,0.007316,0.337,62.460,0.530,65.111,0.259,0.078,4.745
CIP,2019-06-07,3.190,0.001614,0.007995,0.307,60.870,1.379,37.010,0.245,0.062,4.389
GOZ,2019-06-07,4.290,0.001555,0.008101,0.301,60.551,1.591,66.172,0.251,0.050,2.098


In [19]:
# high volume
high_volume = df.sort_values(by='volume_rank', ascending=False).head(20).sort_values(by='return', ascending=False)
print('Highest volume last week performance avg. = %s%%'% week_return_avg(high_volume))
high_volume

Highest volume last week performance avg. = -2.1583%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
FEX,2019-06-07,0.115,0.021486,0.093235,4.226,99.788,96.607,98.727,3.991,0.235,8.696
LTR,2019-06-07,0.105,0.018831,0.094523,3.855,99.576,97.031,99.364,3.442,0.413,0.000
AVH,2019-06-07,0.340,0.014120,0.073468,2.774,98.940,87.275,98.409,3.937,-1.163,-32.353
FMG,2019-06-07,7.850,0.006924,0.028223,1.370,94.486,33.086,99.894,1.377,-0.007,-2.548
MGR,2019-06-07,3.130,0.003223,0.011173,0.621,78.950,3.075,99.576,0.601,0.020,3.195
NEC,2019-06-07,1.965,0.003075,0.022797,0.592,78.049,27.466,98.834,0.872,-0.280,-5.852
AVZ,2019-06-07,0.072,0.002888,0.072546,0.582,77.519,93.425,99.682,0.755,-0.173,-16.667
MPL,2019-06-07,3.350,0.002952,0.017571,0.552,76.246,16.013,98.303,0.584,-0.032,1.194
TLS,2019-06-07,3.720,0.002754,0.012430,0.521,74.761,5.832,100.000,0.495,0.026,1.882


In [20]:
# high return increase in last week
high_return_increase = df.sort_values(by='return_increase',
                                      ascending=False).head(10)
print('Highest return increase last week performance avg. = %s%%'% week_return_avg(high_return_increase))
high_return_increase

Highest return increase last week performance avg. = 18.1547%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
MMM,2019-06-07,0.740,0.010251,0.083116,2.013,97.031,95.970,32.874,0.617,1.396,36.486
K2F,2019-06-07,0.160,0.003968,0.067895,0.781,85.366,91.198,24.708,0.084,0.697,28.125
ASN,2019-06-07,0.059,-0.000821,0.063479,-0.198,21.209,90.986,77.625,-0.892,0.694,18.644
CPH,2019-06-07,0.550,0.001286,0.048784,0.256,58.802,75.080,54.401,-0.426,0.682,23.636
KP2,2019-06-07,0.057,-0.004571,0.090276,-0.888,4.772,99.152,18.558,-1.539,0.651,29.825
NGI,2019-06-07,3.800,0.003651,0.031550,0.684,81.442,46.129,58.431,0.080,0.604,0.263
ZNC,2019-06-07,0.067,-0.000814,0.047234,-0.150,23.542,74.973,12.089,-0.747,0.597,16.418
VMT,2019-06-07,0.115,0.011006,0.093670,2.372,98.621,98.515,64.051,1.803,0.569,13.913
FIJ,2019-06-07,0.078,-0.000095,0.052016,0.086,44.539,79.533,19.194,-0.469,0.555,8.974


In [21]:
# ASX 20
asx_20_list = get_asx_20_list()
asx_20=df[df.index.isin(asx_20_list)].sort_values(by='return', ascending=False)
print('ASX 20 last week performance avg. = %s%%'% week_return_avg(asx_20))
asx_20

ASX 20 last week performance avg. = 0.642%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
TLS,2019-06-07,3.72,0.002754,0.012430,0.521,74.761,5.832,100.000,0.495,0.026,1.882
RIO,2019-06-07,98.00,0.002429,0.014964,0.479,72.481,12.831,83.563,0.553,-0.074,-2.347
BXB,2019-06-07,12.73,0.002111,0.009980,0.405,67.762,2.439,92.895,0.370,0.035,4.478
AMC,2019-06-06,15.77,0.001996,0.008873,0.402,67.603,1.697,93.213,0.438,-0.036,-1.332
TCL,2019-06-07,14.26,0.001962,0.007889,0.380,66.384,0.954,95.228,0.369,0.011,2.314
WES,2019-06-07,37.65,0.001983,0.012165,0.368,65.164,7.211,88.017,0.343,0.025,1.594
BHP,2019-06-07,37.84,0.001644,0.011666,0.323,61.824,6.257,97.349,0.237,0.086,0.370
ANZ,2019-06-07,28.37,0.001462,0.014469,0.297,60.445,13.256,95.970,0.308,-0.011,1.269
CBA,2019-06-07,80.00,0.001318,0.011475,0.249,58.112,6.575,90.032,0.247,0.002,1.862


In [23]:
# highest return in ASX 200
asx_200_list = get_asx_200_list()
asx_200=df[df.index.isin(asx_200_list)].sort_values(by='return', ascending=False).head(20)
print('Top 20 of ASX 200 last week performance avg. = %s%%'% week_return_avg(asx_200))
asx_200

Top 20 of ASX 200 last week performance avg. = -0.0939%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
FMG,2019-06-07,7.85,0.006924,0.028223,1.370,94.486,33.086,99.894,1.377,-0.007,-2.548
APT,2019-06-07,24.17,0.006590,0.039193,1.286,93.955,52.704,85.472,1.276,0.010,0.538
APX,2019-06-07,26.71,0.006893,0.032860,1.245,93.160,44.857,72.216,1.524,-0.279,-4.380
LYC,2019-06-07,2.77,0.006082,0.050779,1.186,91.835,72.534,97.455,1.564,-0.378,-10.108
MFG,2019-06-07,45.83,0.005556,0.016239,1.083,90.986,9.438,65.536,1.195,-0.112,5.237
IEL,2019-06-07,16.71,0.004869,0.025849,0.964,88.865,31.601,69.565,1.017,-0.053,-2.274
BRG,2019-06-07,16.73,0.004874,0.023434,0.933,88.441,24.284,52.174,0.953,-0.020,-1.913
DHG,2019-06-07,3.24,0.004492,0.033820,0.842,86.638,48.462,83.139,0.932,-0.090,-1.543
DLX,2019-06-07,9.74,0.004328,0.028358,0.821,86.320,38.600,82.291,0.854,-0.033,0.103


In [24]:
# high last week return
df.sort_values(by='week_return', ascending=False).head(20).sort_values(by='return', ascending=False)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
VMT,2019-06-07,0.115,0.011006,0.093670,2.372,98.621,98.515,64.051,1.803,0.569,13.913
KSS,2019-06-07,0.265,0.011606,0.051457,2.283,98.197,65.111,62.248,1.758,0.525,16.981
MMM,2019-06-07,0.740,0.010251,0.083116,2.013,97.031,95.970,32.874,0.617,1.396,36.486
BCC,2019-06-07,0.310,0.008017,0.073432,1.423,94.910,92.365,12.619,1.090,0.333,19.355
HFR,2019-06-07,0.935,0.004013,0.046238,0.916,88.123,67.020,28.314,0.431,0.485,21.925
K2F,2019-06-07,0.160,0.003968,0.067895,0.781,85.366,91.198,24.708,0.084,0.697,28.125
AXE,2019-06-07,0.092,0.003106,0.036154,0.578,77.306,52.916,39.873,0.074,0.504,17.391
MML,2019-06-07,0.470,0.002405,0.028197,0.443,70.520,42.948,38.494,0.067,0.376,15.957
CPH,2019-06-07,0.550,0.001286,0.048784,0.256,58.802,75.080,54.401,-0.426,0.682,23.636


## Thisk week recommendations

In [25]:
# recommendations
selection = [
    'FEX',  # high return & high risk
    'ANO',  # high return & high price & low sigma
    'AD8',  # high price & low sigma & low risk
    'MFG',  # high price & low sigma & low risk
    'CHC',  # low sigma & low risk & asx 200
    'JBH',  # low sigma & low risk & asx 200
    'CV1',  # best recent performance
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
FEX,2019-06-07,0.115,0.021486,0.093235,4.226,99.788,96.607,98.727,3.991,0.235,8.696
ANO,2019-06-07,6.480,0.019508,0.068453,3.785,99.470,82.185,25.027,4.132,-0.347,-4.321
CV1,2019-06-07,0.145,0.011898,0.055234,2.377,98.727,70.732,50.901,2.185,0.192,10.345
AD8,2019-06-07,7.760,0.008159,0.025422,1.603,95.652,21.951,40.403,1.531,0.072,0.773
MFG,2019-06-07,45.830,0.005556,0.016239,1.083,90.986,9.438,65.536,1.195,-0.112,5.237
CHC,2019-06-07,10.940,0.003874,0.011683,0.743,83.828,3.393,82.397,0.680,0.063,5.484
JBH,2019-06-07,27.720,0.003262,0.013540,0.655,80.488,7.953,70.201,0.646,0.009,-1.443


## Last week review

In [33]:
selection = [
    'AVH',  # high return & high risk
    'ANO',  # high return & high risk
    'AGH',  # high return & meduim risk
    'Z1P',  # medium price & risk
    'MFG',  # high price & low risk
    'AD8',  # low risk
    'FMG',  # high volume & low risk
]
last_week_df = df[df.index.isin(selection)]
last_week_df.sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return,rank
code,,,,,,,,,,,,
ANO,2019-06-07,6.480,0.019508,0.068453,3.785,99.470,82.185,25.027,4.132,-0.347,-4.321,117.285
AVH,2019-06-07,0.340,0.014120,0.073468,2.774,98.940,87.275,98.409,3.937,-1.163,-32.353,111.665
Z1P,2019-06-07,3.030,0.011510,0.045251,2.310,98.303,56.628,88.547,2.473,-0.163,-5.611,141.675
AGH,2019-06-07,0.655,0.009319,0.046423,1.826,96.607,62.354,60.127,2.012,-0.186,-12.977,134.253
AD8,2019-06-07,7.760,0.008159,0.025422,1.603,95.652,21.951,40.403,1.531,0.072,0.773,173.701
FMG,2019-06-07,7.850,0.006924,0.028223,1.370,94.486,33.086,99.894,1.377,-0.007,-2.548,161.400
MFG,2019-06-07,45.830,0.005556,0.016239,1.083,90.986,9.438,65.536,1.195,-0.112,5.237,181.548


In [35]:
week_return = round(sum(last_week_df['week_return'])/len(selection), 3)
win_rate = round(len(last_week_df[last_week_df['week_return'] >= 0]) / len(selection), 3)
print('Last week average return:', week_return, ' , win rate:', win_rate)

Last week average return: -7.4  , win rate: 0.286


## Signle performance review

In [36]:
def inspect(code):
    print(f'http://h.luotao.net/data/{this_week}/pic/{code}_line.png')
    print(f'http://h.luotao.net/data/{this_week}/pic/{code}.png')
    return df[df.index==code]

In [37]:
inspect('AVH')

http://h.luotao.net/data/20190607/pic/AVH_line.png
http://h.luotao.net/data/20190607/pic/AVH.png


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return,rank
code,,,,,,,,,,,,
AVH,2019-06-07,0.34,0.01412,0.073468,2.774,98.94,87.275,98.409,3.937,-1.163,-32.353,111.665


# Return & Risk Rank

In [41]:
df['rank'] = df['return_rank'] + (100 - df['risk_rank'])
top_df = df.sort_values(by='rank', ascending=False).head(20)
week_return = round(sum(top_df['week_return'].dropna())/len(top_df), 3)
print('Top rank last week average return:', week_return)
top_df

Top rank last week average return: 1.687


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return,rank
code,,,,,,,,,,,,
MFG,2019-06-07,45.83,0.005556,0.016239,1.083,90.986,9.438,65.536,1.195,-0.112,5.237,181.548
CHC,2019-06-07,10.94,0.003874,0.011683,0.743,83.828,3.393,82.397,0.680,0.063,5.484,180.435
MGR,2019-06-07,3.13,0.003223,0.011173,0.621,78.950,3.075,99.576,0.601,0.020,3.195,175.875
AD8,2019-06-07,7.76,0.008159,0.025422,1.603,95.652,21.951,40.403,1.531,0.072,0.773,173.701
DDR,2019-06-07,5.06,0.006426,0.022400,1.245,93.160,19.618,34.783,1.297,-0.052,-1.581,173.542
JBH,2019-06-07,27.72,0.003262,0.013540,0.655,80.488,7.953,70.201,0.646,0.009,-1.443,172.535
ASX,2019-06-07,79.29,0.002558,0.009035,0.495,73.277,1.485,54.719,0.408,0.087,5.057,171.792
ABP,2019-06-07,4.21,0.002520,0.010769,0.497,73.542,3.712,71.580,0.336,0.161,8.314,169.830
TLS,2019-06-07,3.72,0.002754,0.012430,0.521,74.761,5.832,100.000,0.495,0.026,1.882,168.929
